In [ ]:
import numpy as np
import math
from sklearn.model_selection import train_test_split
from scipy.spatial.distance import cdist
from statistics import mode

In [36]:
def sse(distances):
    return np.power(distances, 2).sum()

def euclidean(a, b):
#     FASTER
#     return cdist(a, b, 'euclidean')
    output = []
    for i in a:
        output_inner = []
        for j in b:
            sub = np.subtract(i, j)
            sqr = np.power(sub, 2)
            s = sqr.sum()
            output_inner.append(math.sqrt(s))
        output.append(output_inner)
    return np.array(output)
        
def cosine(a, b):
#     FASTER
#     return cdist(a, b, 'cosine')
    output = []
    for i in a:
        output_inner = []
        for j in b:
            num = np.dot(i, j)
            den = (np.linalg.norm(i) * np.linalg.norm(j))
            
            output_inner.append(1 - num / den)
        output.append(output_inner)
    return np.array(output)

def jaccard(a, b):
#     FASTER
#     return cdist(a, b, 'jaccard')
    output = []
    for i in a:
        output_inner = []
        for j in b:
            num = np.minimum(i, j).sum()
            den = np.maximum(i, j).sum()
            output_inner.append(1 - num / den)
        output.append(output_inner)
    return np.array(output)

In [34]:
def kmeans(X, k, function, stop_at_centroids_same=False, stop_at_sse_greater=False, max_iter=100):
    idx = np.random.choice(len(X), k, replace=False)

    midpoints = X[idx, :]
    dist = function(X, midpoints)
    clusters = np.array([np.argmin(i) for i in dist])
    distance_to_clusters = np.array([np.min(i) for i in dist])
    
    _sse = sse(distance_to_clusters)
    
    early_exit = -1
    
    for i in range(max_iter): 
        new_midpoints = []
        for idx in range(k):
            temp_mid = X[clusters==idx].mean(axis=0) 
            new_midpoints.append(temp_mid)
 
        new_midpoints = np.vstack(new_midpoints) 
        
        if stop_at_centroids_same and (new_midpoints == midpoints).all():
            early_exit = i+1
            break
        
        midpoints = new_midpoints
        dist = function(X, midpoints)
        distance_to_clusters = np.array([np.min(i) for i in dist])
        new_sse = sse(distance_to_clusters)
        
        if stop_at_sse_greater and new_sse > _sse:
            early_exit = i+1
            break
            
        _sse = new_sse
        
        clusters = np.array([np.argmin(i) for i in dist])
    
#     if early_exit < 0:
#         print(max_iter)
#     else:
#         print(early_exit)
    
    return clusters, _sse, (early_exit if early_exit > 0 else max_iter)

In [21]:
import pandas as pd
X = pd.read_csv('/Users/nate/Downloads/hw4_kmeans_data/data.csv', header=None).values
y = pd.read_csv('/Users/nate/Downloads/hw4_kmeans_data/label.csv', header=None)

k = len(y[0].unique())

y = y.values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5)

output_euclidean, euc_sse, _ = kmeans(X_train, k, euclidean, stop_at_centroids_same=True, stop_at_sse_greater=True)
output_cosine, cos_sse, _ = kmeans(X_train, k, cosine, stop_at_centroids_same=True, stop_at_sse_greater=True)
output_jaccard, jac_sse, _ = kmeans(X_train, k, jaccard, stop_at_centroids_same=True, stop_at_sse_greater=True)

In [22]:
# Q1

print(euc_sse)
print(cos_sse)
print(jac_sse)

12725406952.078806
346.36131784121073
4384.295878999801


In [23]:
output_euclidean, euc_sse, _ = kmeans(X_train, k, euclidean, stop_at_centroids_same=True, stop_at_sse_greater=True)
output_cosine, cos_sse, _ = kmeans(X_train, k, cosine, stop_at_centroids_same=True, stop_at_sse_greater=True)
output_jaccard, jac_sse, _ = kmeans(X_train, k, jaccard, stop_at_centroids_same=True, stop_at_sse_greater=True)

In [33]:
# Q2

label_votes_euclidean = [[] for i in range(len(output_euclidean))]
label_votes_cosine = [[] for i in range(len(output_cosine))]
label_votes_jaccard = [[] for i in range(len(output_jaccard))]

labels_euclidean = []
labels_cosine = []
labels_jaccard = []

for i in range(10):
    output_euclidean, euc_sse, _ = kmeans(X_train, k, euclidean, stop_at_centroids_same=True, stop_at_sse_greater=True)
    for j in range(len(output_euclidean)):
        label_votes_euclidean[j].append(output_euclidean[j])
        
for i in range(len(label_votes_euclidean)):
    labels_euclidean.append(mode(np.array(label_votes_euclidean[i])))

for i in range(10):
    output_cosine, cos_sse, _ = kmeans(X_train, k, cosine, stop_at_centroids_same=True, stop_at_sse_greater=True)
    for j in range(len(output_cosine)):
        label_votes_cosine[j].append(output_cosine[j])
        
for i in range(len(label_votes_cosine)):
    labels_cosine.append(mode(np.array(label_votes_cosine[i])))
    
for i in range(10):
    output_jaccard, jac_sse, _ = kmeans(X_train, k, jaccard, stop_at_centroids_same=True, stop_at_sse_greater=True)
    for j in range(len(output_jaccard)):
        label_votes_jaccard[j].append(output_jaccard[j])
        
for i in range(len(label_votes_jaccard)):
    labels_jaccard.append(mode(np.array(label_votes_jaccard[i])))
    
output_euclidean, _ = kmeans(X_train, k, euclidean, stop_at_centroids_same=True, stop_at_sse_greater=True)
output_cosine, _ = kmeans(X_train, k, cosine, stop_at_centroids_same=True, stop_at_sse_greater=True)
output_jaccard, _ = kmeans(X_train, k, jaccard, stop_at_centroids_same=True, stop_at_sse_greater=True)

accuracy_euclidean = 0
accuracy_cosine = 0
accuracy_jaccard = 0

for i in range(len(output_euclidean)):
    if output_euclidean[i] == labels_euclidean[i]:
        accuracy_euclidean += 1
print('Euclidean')
print(accuracy_euclidean / len(output_euclidean))

for i in range(len(output_cosine)):
    if output_cosine[i] == labels_cosine[i]:
        accuracy_cosine += 1
print('Cosine')
print(accuracy_cosine / len(output_cosine))

for i in range(len(output_jaccard)):
    if output_jaccard[i] == labels_jaccard[i]:
        accuracy_jaccard += 1
print('Jaccard:')
print(accuracy_jaccard / len(output_jaccard))

Euclidean
0.214
Cosine
0.1832
Jaccard:
0.047


In [37]:
# Q3

_, _, iter_euclidean = kmeans(X, k, euclidean, stop_at_centroids_same=True, stop_at_sse_greater=True, max_iter=100)
_, _, iter_cosine = kmeans(X, k, cosine, stop_at_centroids_same=True, stop_at_sse_greater=True, max_iter=100)
_, _, iter_jaccard = kmeans(X, k, jaccard, stop_at_centroids_same=True, stop_at_sse_greater=True, max_iter=100)

print(iter_euclidean)
print(iter_cosine)
print(iter_jaccard)

81
39
1


In [ ]:
# Q4
# SSE Comparison

# centroid
output_euclidean, euc_sse, _ = kmeans(X_train, k, euclidean, stop_at_centroids_same=True)
output_cosine, cos_sse, _ = kmeans(X_train, k, cosine, stop_at_centroids_same=True)
output_jaccard, jac_sse, _ = kmeans(X_train, k, jaccard, stop_at_centroids_same=True)
print('CENTROID: ')
print(euc_sse)
print(cos_sse)
print(jac_sse)

# sse increase
output_euclidean, euc_sse, _ = kmeans(X_train, k, euclidean, stop_at_sse_greater=True)
output_cosine, cos_sse, _ = kmeans(X_train, k, cosine, stop_at_sse_greater=True)
output_jaccard, jac_sse, _ = kmeans(X_train, k, jaccard, stop_at_sse_greater=True)
print('\nSSE: ')
print(euc_sse)
print(cos_sse)
print(jac_sse)

# max iterations
output_euclidean, euc_sse, _ = kmeans(X_train, k, euclidean)
output_cosine, cos_sse, _ = kmeans(X_train, k, cosine)
output_jaccard, jac_sse, _ = kmeans(X_train, k, jaccard)
print('\nMAX ITERATIONS')
print(euc_sse)
print(cos_sse)
print(jac_sse)

/var/folders/95/b692db711rb_1p62sdl_s1000000gn/T/ipykernel_802/1937459693.py:16: RuntimeWarning: Mean of empty slice.
  temp_mid = X[clusters==idx].mean(axis=0)


CENTROID: 
12742383100.277157
341.2212853503983
5000.0

SSE: 
12812216123.763365
342.08222951120297
4416.146294118418
